# Non-communicatingなMDPにおける探索と活用について

参考：
* [Near Optimal Exploration-Exploitation in Non-Communicating Markov Decision Processes](https://arxiv.org/abs/1807.02373)

---

前知識（Weakly-communicating MDP）
* MDP: $M=\langle\mathcal{S}, \mathcal{A}, r, p\rangle$
  * $\mathcal{A}=\bigcup_{s \in \mathcal{S}} \mathcal{A}_s$

Weakly-communicating MDPでは，状態空間が２種類に分割されます．
1. Communicatingな状態空間 $\mathcal{S}^C$：$\mathcal{S}^C$の任意の状態が，他の任意の状態から遷移可能であるような方策が存在する．
2. Transientな状態空間 $\mathcal{S}^T$：$s \in \mathcal{S}^T$な$s$について，任意の決定的な定常方策に対して，$s$から再び$s$に戻ってくる確率が$<1$である．（これはつまり，transient stateは有限回しか訪問されないことを意味します（多分一度Transientから出ると，もう戻ってこれない＆CommunicatingからTransientには渡れない？）．また，MDPの半径が$D=+\infty$になります）

補足：今回の話はMulti-chain MDPってやつに簡単にスケールします．TODOにしておきます．

---

今回は無限ホライゾン平均報酬を考えます．
無限ホライゾン平均報酬を解く有名なアルゴリズムとしてUCRLアルゴリズムがあります（https://arxiv.org/abs/1205.2661 など参照）．
UCRLアルゴリズムなどは，MDPの半径が$D < \infty$であることを仮定します．
これはつまり，任意の状態から他の状態に有限ステップ以内に到達できることを仮定していますが，このような半径の仮定は実践では成り立たないことが多いです
（論文中のMountain Carの図参照）．

Weakly communicating MDPなどでは，UCRLアルゴリズムは到達不可能な状態をずっと探そうとしてしまいます．
半径の代わりにspanが$H$でバウンドされていることを利用するアルゴリズム（REGAL.Cなど）も存在しますが，これも到達不可能な状態を排除していることに他なりません．
また，spanのバウンドの事前知識や半径の事前知識を持つことは現実的ではありません．
ちなみに有限ホライゾンの場合でもweakly communicating MDPで生じる「到達不可能な状態をずっと探そうとする」問題は起こりえません．

今回は，到達不可能な状態空間についての事前知識がない場合のアルゴリズムについて見てみます．


---

準備：
* 平均報酬：$g_M^\pi(s):=\lim _{T \rightarrow \infty} \mathbb{E}\left[\frac{1}{T} \sum_{t=1}^T r\left(s_t, \pi\left(s_t\right)\right)\right]$
* バイアス関数：$h_M^\pi(s):=\underset{T \rightarrow \infty}{C\text{-}\lim _T} \mathbb{E}\left[\sum_{t=1}^T\left(r\left(s_t, \pi\left(s_t\right)\right)-g_M^\pi\left(s_t\right)\right)\right]$
    * バイアスを使うと，$h_M^\pi(s)-h_M^\pi\left(s^{\prime}\right)$によって，状態$s$から始まった場合，状態$s'$よりもどれくらい優れているか（劣っているか）測定できます．
* 特に，$s p_{\mathcal{S}}\left\{h^\pi\right\}:=\max _{s \in \mathcal{S}} h^\pi(s)-\min _{s \in \mathcal{S}} h^\pi(s)$のことをバイアス関数のスパンと呼びます．
* Weakly communicating MDPでは，任意の最適方策$\pi^* \in \arg \max _\pi g^\pi(s) l$は定数ゲインを持ちます．つまり，$g^{\pi^*}(s)=g^*$が任意の$s$で成立します．
* 以降，$D$をMDPの半径，$D^C$をCommunicatingな部分の半径とします．つまり，$\tau_M\left(s \rightarrow s^{\prime}\right)$をMDP $M$において$s$から$s'$に到達するまでにかかる期待ステップ数として，以下で定義されます．
$$
D:=\max _{\left(s, s^{\prime}\right) \in \mathcal{S} \times \mathcal{S}, s \neq s^{\prime}}\left\{\tau_M\left(s \rightarrow s^{\prime}\right)\right\}, \quad D^{\mathrm{C}}:=\max _{\left(s, s^{\prime}\right) \in \mathcal{S}^{\mathrm{C}} \times \mathcal{S}^{\mathrm{C}}, s \neq s^{\prime}}\left\{\tau_M\left(s \rightarrow s^{\prime}\right)\right\},
$$
* リグレット：$\Delta(\mathfrak{A}, T)=T g^*-\sum_{t=1}^T r_t\left(s_t, a_t\right)$.
    * ここで$\mathcal{U}$は何らかのアルゴリズムです．
* 仮定：$s_1$はcommunicating な$\mathcal{S}^C$に入っているとします．
* 仮定：状態空間と行動空間，$r_{\max}$は既知とします．それ以外は未知です．



## TUCRLアルゴリズム

TUCRLアルゴリズムはWeakly communicating MDPにおいて，事前情報を必要としないアルゴリズムです．

用語：
* エピソード$k$：これはUCRLにも出てくる概念っぽいですね．エピソードは最低１ステップ進んだ後，次のどちらかが発生した場合に終了します．
    * 今まで見たことない状態に訪れる
    * エピソード開始時までに訪れた回数$N(s, a)$を再びそのエピソードで訪問する．

![TUCRL](figs/TUCRL.png)


TUCRLでは，各エピソードの最初に次の信頼区間を作ります．
$$
\begin{aligned}
B_{p, k}(s, a) & =\left\{\widetilde{p}(\cdot \mid s, a) \in \mathcal{C}: \forall s^{\prime} \in \mathcal{S},\left|\widetilde{p}\left(s^{\prime} \mid s, a\right)-\widehat{p}\left(s^{\prime} \mid s, a\right)\right| \leq \beta_{p, k}^{s a s^{\prime}}\right\} \\
B_{r, k}(s, a) & :=\left[\widehat{r}_k(s, a)-\beta_{r, k}^{s a}, \widehat{r}_k(s, a)+\beta_{r, k}^{s a}\right] \cap\left[0, r_{\max }\right],
\end{aligned}
$$

ここで$\mathcal{C}$は確率単体であり，それぞれのボーナスは
$$
\beta_{r, k}^{s a}:=\sqrt{\frac{14 \widehat{\sigma}_{r, k}^2(s, a) b_{k, \delta}}{N_k^{+}(s, a)}}+\frac{\frac{49}{3} r_{\max } b_{k, \delta}}{N_k^{ \pm}(s, a)}, \quad \beta_{p, k}^{s a s^{\prime}}:=\sqrt{\frac{14 \widehat{\sigma}_{p, k}^2\left(s^{\prime} \mid s, a\right) b_{k, \delta}}{N_k^{+}(s, a)}}+\frac{\frac{49}{3} b_{k, \delta}}{N_k^{ \pm}(s, a)},
$$

とします．また，$N_k^{+}(s, a):=\max \left\{1, N_k(s, a)\right\}$と$N_k^{ \pm}(s, a):=\max \left\{1, N_k(s, a)-1\right\}$です．
以降，$\mathcal{M}_k=\left\{M=(\mathcal{S}, \mathcal{A}, \widetilde{r}, \widetilde{p}): \widetilde{r}(s, a) \in B_{r, k}(s, a), \widetilde{p}(\cdot \mid s, a) \in B_{p, k}(s, a)\right\}$とします．

さて，UCRLアルゴリズムはこの$\mathcal{M}$について楽観的に方策を選択するわけですが，この場合，到達不可能な状態空間をずっと探索しかねません．
これを回避するために，TUCRLでは最初に$\mathcal{S}^C$と$\mathcal{S}^T$を推定して構築します．具体的には，

$$
\mathcal{S}_k^{\mathrm{C}}:=\left\{s \in \mathcal{S} \mid \sum_{a \in \mathcal{A}_s} N_k(s, a)>0\right\} \cup
$$
および
$$
\mathcal{S}_k^{\mathrm{T}}:=\mathcal{S} \backslash \mathcal{S}_k^{\mathrm{C}}
$$
とします．
これを使って，TUCRLでは$\mathcal{S}^C_k$だけを探索することを考えます．
しかし，このままでは$\mathcal{S}^C$にあるが，$\mathcal{S}^C_k$にはない状態を探索することができません．そこで，$\mathcal{S}^C_k$から$\mathcal{S}^T_k$に遷移する確率の下界を考え，もし$s$から$s'$へ行く確率がその下界以下であれば，その遷移は無視します．

$\mathcal{S}^T_k$へ行く遷移は存在しないので，OptimisticなMDPにおける遷移は最大でも$\tilde{p}_k^{+}\left(s^{\prime} \mid s, a\right)=\frac{49}{3} \frac{b_{k, \delta}}{N_k^{ \pm}(s, a)}$になります．
この値が，後で決める$\rho_{t_k}$以下であれば，その遷移は無視することにします．
よって，次のような信頼区間を考えましょう．

$$
\bar{B}_{p, k}(s, a)=B_{p, k}(s, a) \cap\left\{\widetilde{p}(\cdot \mid s, a) \in \mathcal{C}: \forall s^{\prime} \in \mathcal{S}_k^{\mathrm{T}} \text { and } \widetilde{p}_k^{+}\left(s^{\prime} \mid s, a\right)<\rho_{t_k}, \widetilde{p}\left(s^{\prime} \mid s, a\right)=0\right\}
$$

TUCRLはこの信頼区間に対してOptimisticな方策を選択します．

$$
\left(\widetilde{M}_k, \widetilde{\pi}_k\right)=\underset{M \in \overline{\mathcal{M}}_k, \pi}{\arg \max }\left\{g_M^\pi\right\}
$$

また，
$$
\rho_t=\frac{49 b_{t, \delta}}{3} \sqrt{\frac{S A}{t}}
$$
とします．つまり，$N_k^{ \pm}(s, a)>\sqrt{t_k / S A}$である場合は$s'$を無視します．

## リグレット解析

TUCRLは高確率で次のリグレットを達成します：

$$
\Delta(\mathrm{TUCRL}, T)=O\left(r_{\max } D^c \sqrt{\Gamma^c S^c A T \ln \left(\frac{S A T}{\delta}\right)}+r_{\max }\left(D^c\right)^2 S^3 A \ln ^2\left(\frac{S A T}{\delta}\right)\right)
$$

これを示しましょう．

### ステップ１：リグレットの分解

リグレットは$\Delta(\mathrm{TUCRL}, T):=T g^*-\sum_{t=1}^T r_t\left(s_t, a_t\right)$でした．$\Delta_k=\sum_{s \in \mathcal{S}, a \in \mathcal{A}} \nu_k(s, a)\left(g^*-r(s, a)\right)$とすると（ここで$\nu_k(s, a)$はエピソード$k$中で訪問した$(s, a)$の回数です），

TODO: ここから先は多分UCRLを知らないと書けない．

